In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Şimdilik, veriyi direkt olarak 'data' klasöründen okuyormuş gibi varsayalım.
try:
    df = pd.read_csv("C:/Users/safye/Desktop/Customer_Churn_CLV_Project/Customer_Churn_CLV_Project/data/OnlineRetail.csv/OnlineRetail.csv", encoding = 'unicode_escape') 
    print("Veri başarıyla yüklendi.")
    df_copy = df.copy() 
except FileNotFoundError:
    print("HATA: Lütfen dosyanın 'data/Online_Retail.csv' yolunda olduğundan emin olun.")

# İlk kontrolleri tekrar yapalım:
print("\n--- Eksik Değer Kontrolü (isnull) ---")
print(df.isnull().sum())

Veri başarıyla yüklendi.

--- Eksik Değer Kontrolü (isnull) ---
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [ ]:
# Temizlik ve Dönüşümler

# 1. Eksik Müşteri ID'lerini Silme
# Toplam 135,080 satırı siliyoruz.
print(f"Başlangıç Müşteri ID NaN Sayısı: {df['CustomerID'].isnull().sum()}")
df.dropna(subset=['CustomerID'], inplace=True) 

Başlangıç Müşteri ID NaN Sayısı: 0


Neden İptal Edilen İşlemleri Çıkarıyoruz

1= Monetary (M) İçin: 
Müşteri 100 TL'lik bir alışveriş yaptı, sonra bu siparişi iptal etti. Eğer iptal edilen bu işlemi veri setinde bırakırsak, müşterinin toplam harcaması (M) hatalı olarak 100 TL daha fazla görünür. Modelimiz, aslında parayı geri almış bir müşteriyi yüksek değerli sanır.

2= Frequency (F) İçin:
 Aynı şekilde, iptal edilen bir sipariş, müşterinin toplam sipariş sayısını (F) hatalı olarak artırır. Model, aslında vazgeçmiş bir müşteriyi sadık sanabilir.Bu yüzden bu satırları çıkararak sadece tamamlanmış ve ödenmiş gerçek işlemleri saklıyoruz.

In [6]:
# 2. İptal Edilen İşlemleri Çıkarma
df = df[~df["InvoiceNo"].astype(str).str.contains("C", na=False)] 

# 3. Harcama Miktarı Sıfır ve Negatif Olanları Çıkarma
# Hata olarak girilmiş veya anlamsız (sıfır, negatif) miktarları ve fiyatları veri setinden temizleriz.
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# 4. Veri Tipi Dönüşümleri
df["CustomerID"] = df["CustomerID"].astype(int) 

# 5. Monetary (Harcama) Değişkenini Oluşturma (M)
# Müşteri Harcaması = Miktar * Birim Fiyat
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

# 6. InvoiceDate (Tarih) Tipini Dönüştürme
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

print("--- Temizlik Başarıyla Tamamlandı ---")
print(f"Toplam Kalan Satır Sayısı: {len(df)}")
print(f"Toplam Tekil Müşteri Sayısı: {df['CustomerID'].nunique()}")

--- Temizlik Başarıyla Tamamlandı ---
Toplam Kalan Satır Sayısı: 397884
Toplam Tekil Müşteri Sayısı: 4338
